### Surface Code Ancilla Channel Optimization


In [1]:
from src.game import simulate, Game

import networkx as nx
import numpy as np
import mlrose_ky as mlrose

import random
import math

#### Defining Parameters and Workload

In [37]:
# Number of points 
n = 16
min_x = 0
max_x = 10
min_y = 0
max_y = 10

# Define topology
topo = nx.grid_2d_graph(max_x, max_y)

# Define workload 
l = list(range(n))
random.seed(0)
random.shuffle(l)
size = len(l)
if size % 2 == 1:
    size -= 1

first_l = l[:size // 2]
last_l = l[size // 2:]
layer = list(zip(first_l, last_l))

workload = [layer] * 10

error_rate = 0.01

n_trials = 1000


#### Optimization Methods
- Mesh
- Random Search
- Simulated Annealing

##### Mesh

In [38]:
locs = []
x_range = np.linspace(min_x, max_x, num=int(math.sqrt(n) + 1), endpoint=False)
y_range = np.linspace(min_y, max_y, num=int(math.sqrt(n) + 1), endpoint=False)
for x in x_range[1:]:
    for y in y_range[1:]:
        locs.append((x,y))

print(locs)

success_rate, throughput, _ = simulate(topo, locs, workload, lambda_=error_rate, n=n_trials)

print(f"Success Rate: {success_rate}, Throughput: {throughput}")

[(2.0, 2.0), (2.0, 4.0), (2.0, 6.0), (2.0, 8.0), (4.0, 2.0), (4.0, 4.0), (4.0, 6.0), (4.0, 8.0), (6.0, 2.0), (6.0, 4.0), (6.0, 6.0), (6.0, 8.0), (8.0, 2.0), (8.0, 4.0), (8.0, 6.0), (8.0, 8.0)]
Success Rate: 0.279, Throughput: 0.621380846325167


##### Random Search

In [4]:
iterations = 1000

def sample_unique_coordinates(x_min, x_max, y_min, y_max, n):
    if n > (x_max - x_min + 1) * (y_max - y_min + 1):
        raise ValueError("n is too large for the given range, cannot generate unique points.")
    
    all_points = [(x, y) for x in range(x_min, x_max) for y in range(y_min, y_max)]
    return random.sample(all_points, n)

highest_throughput = 0.0
throughput_points = None
highest_success_rate = 0.0
success_rate_points = None

for i in range(iterations):
    print("Running {}/{}".format(i+1, iterations))
    locs = sample_unique_coordinates(min_x, max_x, min_y, max_y, n)

    success_rate, throughput, game = simulate(topo, locs, workload, lambda_=error_rate, n=n_trials)

    if success_rate > highest_success_rate:
        highest_success_rate = success_rate
        success_rate_points = locs
    
    if throughput > highest_throughput:
        highest_throughput = throughput
        throughput_points = locs


Running 1/1000


Running 2/1000
Running 3/1000
Running 4/1000
Running 5/1000
Running 6/1000
Running 7/1000
Running 8/1000
Running 9/1000
Running 10/1000
Running 11/1000
Running 12/1000
Running 13/1000
Running 14/1000
Running 15/1000
Running 16/1000
Running 17/1000
Running 18/1000
Running 19/1000
Running 20/1000
Running 21/1000
Running 22/1000
Running 23/1000
Running 24/1000
Running 25/1000
Running 26/1000
Running 27/1000
Running 28/1000
Running 29/1000
Running 30/1000
Running 31/1000
Running 32/1000
Running 33/1000
Running 34/1000
Running 35/1000
Running 36/1000
Running 37/1000
Running 38/1000
Running 39/1000
Running 40/1000
Running 41/1000
Running 42/1000
Running 43/1000
Running 44/1000
Running 45/1000
Running 46/1000
Running 47/1000
Running 48/1000
Running 49/1000
Running 50/1000
Running 51/1000
Running 52/1000
Running 53/1000
Running 54/1000
Running 55/1000
Running 56/1000
Running 57/1000
Running 58/1000
Running 59/1000
Running 60/1000
Running 61/1000
Running 62/1000
Running 63/1000
Running 64/1000


In [5]:
# Print out the results
print ("Success Rate")
print("Best Points:", success_rate_points)
print("Max Value:", highest_success_rate)

print ("Throughput")
print("Best Points:", throughput_points)
print("Max Value:", highest_throughput)

Success Rate
Best Points: [(4, 6), (5, 7), (8, 4), (6, 3), (0, 6), (3, 5), (7, 2), (8, 6), (3, 4), (2, 5), (2, 2), (7, 6), (7, 1), (3, 2), (6, 5), (1, 1)]
Max Value: 0.454
Throughput
Best Points: [(2, 3), (3, 5), (6, 0), (5, 4), (2, 6), (9, 1), (6, 3), (7, 2), (8, 5), (5, 6), (4, 8), (0, 0), (1, 8), (4, 6), (7, 1), (7, 8)]
Max Value: 0.9067633880947301


##### Simulated Annealing

In [39]:
# Define parameters

# Define optimization functions
def opt_success_rate(state):
    locs = list(zip(state[::2], state[1::2]))
    if len(set(locs)) < len(locs):
        return -1e6
    success_rate, _, _= simulate(topo, locs, workload, lambda_=error_rate, n=n_trials)
    return success_rate

def opt_throughput(state):
    locs = list(zip(state[::2], state[1::2]))
    if len(set(locs)) < len(locs):
        return -1e6
    _, throughput, _ = simulate(topo, locs, workload, lambda_=error_rate, n=n_trials)
    return throughput



In [40]:
# Define optimization problem
fitness = mlrose.CustomFitness(lambda state: opt_success_rate(state))
problem = mlrose.DiscreteOpt(2*n, fitness, max_val=max_x)

# Run simulated anneaing
best_state, best_fitness, _ = mlrose.simulated_annealing(
    problem,
    schedule=mlrose.ExpDecay(),
    max_attempts=10,
    max_iters=100
)

In [41]:
# Get results
best_points = list(zip(best_state[::2], best_state[1::2]))
print ("Optimizing for Success Rate")
print("Best Points:", best_points)
print("Max Value:", best_fitness)

Optimizing for Success Rate
Best Points: [(6, 3), (9, 6), (9, 4), (2, 6), (1, 8), (0, 7), (7, 7), (0, 2), (1, 2), (3, 4), (2, 7), (6, 4), (4, 0), (2, 5), (1, 5), (5, 2)]
Max Value: 0.078


In [43]:
# Define optimization problem
fitness = mlrose.CustomFitness(lambda state: opt_throughput(state))
problem = mlrose.DiscreteOpt(2*n, fitness, max_val=max_x)

# Run simulated anneaing
best_state, best_fitness, _ = mlrose.simulated_annealing(
    problem,
    schedule=mlrose.ExpDecay(),
    max_attempts=10,
    max_iters=100
)

In [44]:
# Get results
best_points = list(zip(best_state[::2], best_state[1::2]))
print ("Optimizing for Throughput")
print("Best Points:", best_points)
print("Max Value:", best_fitness)

Optimizing for Throughput
Best Points: [(5, 9), (1, 6), (2, 7), (7, 0), (1, 3), (5, 8), (6, 2), (9, 7), (1, 4), (5, 6), (8, 8), (7, 1), (8, 4), (6, 5), (6, 4), (8, 9)]
Max Value: 0.5470085470085471


#### Figures

In [4]:
import matplotlib.pyplot as plt

def draw_lattice(topo: nx.Graph, qubit_to_data: dict[int, tuple[int, list[tuple]]], data_to_qubit, file_name: str) -> None:
    plt.figure(figsize=(8, 8))
    pos = {(x, y): (y, -x) for x, y in topo.nodes()} 
    nx.draw(topo, pos=pos, with_labels=False, node_size=300, 
            node_color="skyblue", font_size=5, font_color="black")
    
    # node_labels = nx.get_node_attributes(self.topo, 'qubit')
    # nx.draw_networkx_labels(self.topo, pos, labels=node_labels, font_size=2)

    # edge_labels = nx.get_edge_attributes(self.topo, 'cnot_control')
    # nx.draw_networkx_edge_labels(self.topo, pos, edge_labels=edge_labels, font_size=2)

    for k,v in qubit_to_data.items():
        subgraph = nx.subgraph(topo, v[1])
        nx.set_node_attributes(subgraph, [v[0]] * len(v[1]), "label")
        nx.draw(subgraph, pos=pos, with_labels=True, labels=data_to_qubit, 
                node_size=300, node_color="orange", 
                font_size=5, font_color="black")

    plt.savefig(file_name, dpi = 600)
    plt.close()

In [5]:
# Code Expansion
# Number of points 
n = 16
min_x = 0
max_x = 20
min_y = 0
max_y = 20

# Define topology
topo = nx.grid_2d_graph(max_x, max_y)

# Define workload 
l = list(range(n))
random.seed(0)
random.shuffle(l)
size = len(l)
if size % 2 == 1:
    size -= 1

first_l = l[:size // 2]
last_l = l[size // 2:]
layer = list(zip(first_l, last_l))

workload = [layer] * 3

error_rate = 0.05

n_trials = 1


locs = []
x_range = np.linspace(min_x, max_x, num=int(math.sqrt(n) + 1), endpoint=False)
y_range = np.linspace(min_y, max_y, num=int(math.sqrt(n) + 1), endpoint=False)
for x in x_range[1:]:
    for y in y_range[1:]:
        locs.append((x,y))


g = Game(topo, locs, workload, lambda_=error_rate)

draw_lattice(topo, g.qubit_to_data_, g.data_to_qubit_, "/home/peizhi/Projects/EDN/ancilla_channels_sim/figures/before_expand.png")

print(g.run())

draw_lattice(g.topology_, g.qubit_to_data_, g.data_to_qubit_, "/home/peizhi/Projects/EDN/ancilla_channels_sim/figures/after_expand.png")


(True, 4)


In [ ]:
# Number of points 
n = 16
min_x = 0
max_x = 10
min_y = 0
max_y = 10

# Define topology
topo = nx.grid_2d_graph(max_x, max_y)


mesh_points = [(2.0, 2.0), (2.0, 4.0), (2.0, 6.0), (2.0, 8.0), (4.0, 2.0), (4.0, 4.0), (4.0, 6.0), (4.0, 8.0), (6.0, 2.0), (6.0, 4.0), (6.0, 6.0), (6.0, 8.0), (8.0, 2.0), (8.0, 4.0), (8.0, 6.0), (8.0, 8.0)]
g = Game(topo, mesh_points, workload, lambda_=error_rate)
draw_lattice(topo, g.qubit_to_data_, g.data_to_qubit_, "/home/peizhi/Projects/EDN/ancilla_channels_sim/figures/mesh_points.png")

mesh_points = [(4, 6), (5, 7), (8, 4), (6, 3), (0, 6), (3, 5), (7, 2), (8, 6), (3, 4), (2, 5), (2, 2), (7, 6), (7, 1), (3, 2), (6, 5), (1, 1)]
g = Game(topo, mesh_points, workload, lambda_=error_rate)
draw_lattice(topo, g.qubit_to_data_, g.data_to_qubit_, "/home/peizhi/Projects/EDN/ancilla_channels_sim/figures/best_succ_points.png")

mesh_points = [(2, 3), (3, 5), (6, 0), (5, 4), (2, 6), (9, 1), (6, 3), (7, 2), (8, 5), (5, 6), (4, 8), (0, 0), (1, 8), (4, 6), (7, 1), (7, 8)]
g = Game(topo, mesh_points, workload, lambda_=error_rate)
draw_lattice(topo, g.qubit_to_data_, g.data_to_qubit_, "/home/peizhi/Projects/EDN/ancilla_channels_sim/figures/best_throughput_points.png")


